# Modeling
In this section, we train and evaluate regression models to predict the SalePrice of homes using the features we selected and preprocessed in the previous notebook.

The model will use the following features:
- `TotalSF` – Total home area (square feet)
- `Overall Qual` – Overall material and finish quality
- `Garage Cars` – Garage capacity (number of cars)
- `HouseAge` – Year of original construction

The target variable is:
- `SalePrice` – Final selling price of the property

## Load Data

In [7]:
import joblib

# Load split and scaled data
X_train = joblib.load("../data/X_train_scaled.pkl")
X_test = joblib.load("../data/X_test_scaled.pkl")
y_train = joblib.load("../data/y_train.pkl")
y_test = joblib.load("../data/y_test.pkl")